# Unsupervised learning Capstone (name TBA)
Author: Matthew Huh
    
## Overview

For the most part, people are free to choose what news outlets they read and follow. In the United States, there is a near-endless list of sites that people can choose from in order to get their daily news and over time, they develop preferences for sites that they are more attached to, and do their best to avoid. Now these affinities are developed through a combination of means ranging from affiliations, vocabulary, prose, and so forth.

What I would like to examine in this project is if it is possible to differentiate from several different publications with their respective perks / quirks. 

## About the Data

This dataset was obtained from Kaggle, and contains a collection of 142,570 articles from 15 different publications.

The publications within this dataset are
1. CNN
2. Breitbart
3. Vox
4. Washington Post
5. New York Post
6. National Review
7. NPR
8. Guardian
9. Talking Points Memo
10. Atlantic
11. Reuters
12. Fox News
13. Business Insider
14. Buzzfeed News
15. New York Times

## Research Question

As this is an unsupervised learning project first and foremost, the project will have 3 goals.

1. The first goal is to prepare the articles in the dataset for modelling using various Natural Language Processing (NLP) methods to re-represent the data in numbers rather than words
2. Cluster the data to determine if we can identify the articles and associate them as different groups.
3. Determine if we can predict the structure of the article based on the publisher.

## Packages

In [1]:
# Basic imports
import os
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Machine Learning packages
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import chi2
from sklearn.preprocessing import normalize
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

# Clustering packages
import sklearn.cluster as cluster
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AffinityPropagation
from scipy.spatial.distance import cdist

# Natural Language processing
import re
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_rcv1

## Data Preview

The first matter of business is to import the articles from a local directory and merge them.

In [2]:
# Create list of files from directory
filelist = os.listdir('articles')

# Import the files
df_list = [pd.read_csv(file) for file in filelist]

#concatenate them together
articles = pd.concat(df_list)

# Preview the data
articles.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [3]:
# Print the size of the dataset
articles.shape

(142570, 10)

So we have 142,570 articles in the dataset but unfortunately, NLP is quite memory intensive, so we will have to sample the dataset unless you happen to have over 120 GB of memory on your local device. Using a 10% sample still leaves us with 140,000 articles and will be used for the duration of this project.

In [4]:
# Sample the dataset for optimal performance
articles = articles.sample(frac=0.1)

In [5]:
# Print out unique publisher names
articles.publication.unique()

array(['Business Insider', 'New York Post', 'Fox News', 'CNN',
       'New York Times', 'Breitbart', 'Talking Points Memo', 'NPR',
       'Reuters', 'Guardian', 'Washington Post', 'Vox', 'Atlantic',
       'Buzzfeed News', 'National Review'], dtype=object)

In [6]:
# Describe unique occurences for each categorical variable
articles.select_dtypes(include=['object']).nunique()

title          14247
publication       15
author          3902
date            1040
url             8609
content        14244
dtype: int64

There are also other ways to trim down the dataset before processing. We aren't particularly interested in examining the dates for this research question, but it may be of interest in another. Let's check to see how many articles each author wrote; it may not be very useful to examine authors that are only responsible for a single article, as different authors from the same publisher may choose compose their works differently.

In [7]:
# Drop variables that have no impact on the outcome
articles = articles[['title', 'publication', 'author', 'content']]

In [8]:
# View most frequently occurring authors
articles.groupby(['author']).size().sort_values(ascending=False)

author
Pam Key                                    133
Breitbart News                             129
Associated Press                           126
Jerome Hudson                               85
Daniel Nussbaum                             84
Charlie Spiering                            78
John Hayward                                72
Ian Hanchett                                71
Camila Domonoske                            70
Post Editorial Board                        70
AWR Hawkins                                 68
Joel B. Pollak                              65
Trent Baker                                 57
NPR Staff                                   57
Warner Todd Huston                          51
Alex Swoyer                                 49
Reuters                                     47
Josh Marshall                               44
Jeff Poor                                   44
Katherine Rodriguez                         43
Breitbart London                            43
Esme C

Well, that partly explains how there are so many authors in this dataset. It seems as though there are over 15,000 authors, and many of them have only published one article, or have co-written multiple articles with other authors. This complicates the problem, so in order to best represent each author's writing style, let's see what happens if we simply remove all authors that only published one article as is.

In [40]:
# Plotly packages
import plotly as py
import plotly.graph_objs as go
from plotly import tools
import cufflinks as cf
import ipywidgets as widgets
from scipy import special
py.offline.init_notebook_mode(connected=True)

# Pass in values for our pie chart
trace = go.Pie(labels=articles['publication'].unique(), values = articles['publication'].value_counts())

# Create the layout
layout = go.Layout(
    title = 'Articles by Publication',
    height = 600,
    width = 800,
    autosize = False
)

# Construct the chart
fig = go.Figure(data = [trace], layout = layout)
py.offline.iplot(fig, filename ='cufflinks/simple')

## Feature Selection

In [9]:
# Drop author from the dataframe if they wrote less than 5 articles
vc = articles['author'].value_counts()
u  = [i not in set(vc[vc<=4].index) for i in articles['author']]
articles = articles[u]

In [10]:
# Reprint how many unique authors there are
articles.select_dtypes(include=['object']).nunique()

title          9318
publication      15
author          589
content        9317
dtype: int64

In [11]:
# View number of articles after feature selection
articles.shape

(9327, 4)

So after removing authors that composed fewer than 5 articles, we are left with 9k articles, or 67% of the data, and roughly 600/3900 of the authors. Now, we can create a better representation of each author since each author has at least 5 articles to evaluate from.

## Text Cleaning

Now that we've chosen which articles to use, it's time to clean them up and prepare them for feature engineering. What this section covers is the removal of annoying punctuation from the content, and reducing words to their lemmas to reduce the number of words that we are examining. Finally, we'll divide the articles into training and testing sets and separate our predictor, the words in the content, and the target, the publisher.

In [12]:
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

In [13]:
# Remove annoying punctuation from the articles
articles['content'] = articles.content.map(lambda x: text_cleaner(str(x)))
articles.head()

,title,publication,author,content
47986,Donald Trump officially endorses Paul Ryan aft...,Business Insider,Bryan Logan,’ ’ ’ Donald Trump has officially thrown his s...
11168,ISIS militants re-enter Syria’s historic Palmyra,Fox News,NaN,The Palmyra Coordination network said the mili...
10886,’Don’t tell me’: Georgia man deliberately stay...,Fox News,NaN,“I was invited to an election party to stay up...
38584,Possible Ebola exposure in Canadian health lab,CNN,NaN,(CNN) An employee at a Canadian infectious dis...
42617,GOP hopefuls split in reactions to same-sex ma...,CNN,Tom LoBianco,Washington (CNN) Republicans seeking the White...


In [14]:
lemmatizer = WordNetLemmatizer()

# Reduce all text to their lemmas
for article in articles['content']:
    article = lemmatizer.lemmatize(article)

In [15]:
# Identify predictor and target variables
X = articles['content']
y = articles['publication']

# Create training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

### Tf-idf Vectorization

The first types of features that we are going to add are the most useful words in our dataset. Now how are we going to determine which words are deemed the most "useful"? With TF-IDF vectorizer, of course.

TF tracks the term frequency, or how often each word appears in all articles of text, while idf (or Inverse Document Frequency) is a value that places less weight on variables that occur too often and lose their predictive power. Put together, it's a tool that allows us to assign an importance value to each word in the entire dataset based on frequency in each row and throughout the database.

These are the parameters that will be used for TF-IDF
1. All words that appear in over half of the articles will be thrown out of the dataframe
2. Only words that occur more than 5 times will be tracked
3. Only the top 150 features (words) will be kept
4. Stop words will be ignored (like, as, the)
5. Cases will be ignored
6. Shorter and longer articles will be treated equally
7. Add 1 to document frequency in case we have to divide by 0

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Parameters for TF-idf vectorizer
vectorizer = TfidfVectorizer(max_df=0.5,
                             min_df=5, 
                             max_features=150, 
                             stop_words='english', 
                             lowercase=True, 
                             use_idf=True,
                             norm=u'l2',
                             smooth_idf=True
                            )

#Applying the vectorizer
X_tfidf=vectorizer.fit_transform(X)
print("Number of features: %d" % X_tfidf.get_shape()[1])

#splitting into training and test sets
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.25, random_state=42)

#Removes all zeros from the matrix
X_train_tfidf_csr = X_train_tfidf.tocsr()

#number of paragraphs
n = X_train_tfidf_csr.shape[0]

#A list of dictionaries, one per paragraph
tfidf_bypara = [{} for _ in range(0,n)]

#List of features
terms = vectorizer.get_feature_names()

#for each paragraph, lists the feature words and their tf-idf scores
for i, j in zip(*X_train_tfidf_csr.nonzero()):
    tfidf_bypara[i][terms[j]] = X_train_tfidf_csr[i, j]

# Normalize the dataset    
X_norm = normalize(X_train_tfidf)

# Convert from tf-idf matrix to dataframe
X_normal  = pd.DataFrame(data=X_norm.toarray())

Number of features: 150


### Phrase count with spacy

The second set of variables that we will be creating are counters of how often each publishers makes use of each part of speech, meaning adverbs, verbs, nouns, adjectives, as well as article length.

In [18]:
# Instantiating spaCy
nlp = spacy.load('en')
X_train_words = []

for row in X_train:
    # Processing each row for tokens
    row_doc = nlp(row)
    # Calculating length of each sentence
    sent_len = len(row_doc) 
    # Initializing counts of different parts of speech
    advs = 0
    verb = 0
    noun = 0
    adj = 0
    for token in row_doc:
        # Identifying each part of speech and adding to counts
        if token.pos_ == 'ADV':
            advs +=1
        elif token.pos_ == 'VERB':
            verb +=1
        elif token.pos_ == 'NOUN':
            noun +=1
        elif token.pos_ == 'ADJ':
            adj +=1
    # Creating a list of all features for each sentence
    X_train_words.append([row_doc, advs, verb, noun, adj, sent_len])

# Create dataframe with count of adverbs, verbs, nouns, and adjectives
X_count = pd.DataFrame(data=X_train_words, columns=['BOW', 'ADV', 'VERB', 'NOUN', 'ADJ', 'sent_length'])

# Change token count to token percentage
for column in X_count.columns[1:5]:
    X_count[column] = X_count[column] / X_count['sent_length']

# Normalize X_count
X_counter = normalize(X_count.drop('BOW',axis=1))
X_counter  = pd.DataFrame(data=X_counter)

In [19]:
# Combine tf-idf matrix and phrase count matrix
features = pd.concat([X_counter,X_normal], ignore_index=False, axis=1)
features.head()

,0,1,2,3,4,0,1,2,3,4,...,140,141,142,143,144,145,146,147,148,149
0,0.000126,0.000504,0.000511,0.000161,1.0,0.000000,0.0,0.000000,0.372946,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.170872,0.000000
1,0.000127,0.000463,0.000341,0.000180,1.0,0.000000,0.0,0.000000,0.000000,0.077174,...,0.000000,0.000000,0.0,0.0,0.065274,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000029,0.000344,0.000383,0.000180,1.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.218226,0.233106,0.0,0.0,0.227370,0.111283,0.137396,0.000000,0.000000,0.247051
3,0.000041,0.000225,0.000312,0.000092,1.0,0.094248,0.0,0.000000,0.038672,0.101425,...,0.041168,0.043975,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000030,0.000172,0.000204,0.000048,1.0,0.000000,0.0,0.042325,0.000000,0.000000,...,0.076804,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.034672,0.000000,0.000000


And now we have our list of features. It doesn't look anything like our original dataset, now does it? That's because our sentences have been transformed into numbers to feed into our clustering algorithms and predictive models.

# Clustering

Now it's finally time for some unsupervised machine learning. Each article has been binarized to 1s and 0s, and it's time to determine if we can determine if each publisher has a different method for publication.

### K-means

The first clustering method I'll use for modelling the dataset is K-means, that requires the user to input k number of centroids, determining the nearest centroid for each data point, and adjusting the centroids until the best clusters are found, or until a set number of iterations has passed. However, we want to see if we can cluster the articles into 15 clusters representing each of the publishers, so that will be k.

In [20]:
# Calulate predicted values
kmeans = KMeans(n_clusters=15, init='k-means++', random_state=42, n_init=20)
y_pred = kmeans.fit_predict(features)

pd.crosstab(y_train, y_pred)

col_0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
publication,,,,,,,,,,,,,,,
Atlantic,13,21,25,47,11,32,16,3,25,13,17,13,8,49,93
Breitbart,85,16,182,250,29,173,38,57,107,53,36,89,21,375,122
Business Insider,12,12,29,86,50,30,5,8,26,7,5,17,7,93,55
Buzzfeed News,3,2,3,19,15,34,14,0,10,0,5,25,2,34,22
CNN,30,19,31,89,5,107,20,2,36,0,16,69,22,138,46
Fox News,14,8,40,43,5,46,8,9,11,2,1,38,6,49,13
Guardian,8,7,7,41,11,35,11,0,28,0,9,16,2,54,55
NPR,14,119,17,44,8,43,17,2,28,14,18,23,9,68,91
National Review,30,7,30,61,2,21,12,13,17,15,8,7,6,27,54


In [21]:
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import silhouette_score

print('Adjusted Rand Score: {:0.7}'.format(adjusted_rand_score(y_train, y_pred)))
print('Silhouette Score: {:0.7}'.format(silhouette_score(features, y_pred, sample_size=60000, metric='euclidean')))

Adjusted Rand Score: 0.02422749
Silhouette Score: 0.06944205


Oh, that doesn't look very good now does it? Based on the clustering above, and our scores, it seems as though it's not very effective. Let's see if there is an issue with what we're measuring by assessing other clustering methods first.

### Spectral Clustering

In [22]:
sc = SpectralClustering(n_clusters=15)
y_pred2 = sc.fit_predict(features)

pd.crosstab(y_train, y_pred2)

col_0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
publication,,,,,,,,,,,,,,,
Atlantic,42,49,20,36,114,16,15,9,1,14,11,25,15,7,12
Breitbart,438,245,65,162,157,33,10,64,52,38,48,164,82,18,57
Business Insider,83,51,26,68,70,2,9,13,8,5,7,27,11,5,57
Buzzfeed News,37,44,12,11,22,5,2,21,0,11,0,2,3,2,16
CNN,122,157,38,62,54,12,14,60,2,20,0,30,30,20,9
Fox News,48,65,12,38,16,3,7,31,8,6,0,34,14,5,6
Guardian,50,50,23,33,63,9,4,13,0,10,0,6,7,1,15
NPR,56,74,32,32,108,16,99,15,3,15,12,14,17,7,15
National Review,33,30,14,46,63,7,4,5,12,12,18,29,30,5,2


In [23]:
print('Adjusted Rand Score: {:0.7}'.format(adjusted_rand_score(y_train, y_pred2)))
print('Silhouette Score: {:0.7}'.format(silhouette_score(features, y_pred2, sample_size=60000, metric='euclidean')))

Adjusted Rand Score: 0.02697467
Silhouette Score: 0.05824182


### Affinity Propagation

Now, for our final attempt at clustering, affinity propagation. It's a method that will group like data points, but most likely result in an excessive number of clusters. Let's see if that can work to our advantage.

In [24]:
af = AffinityPropagation()
y_pred3 = af.fit_predict(features)

pd.crosstab(y_train, y_pred3)

col_0,0,1,2,3,4,5,6,7,8,9,...,272,273,274,275,276,277,278,279,280,281
publication,,,,,,,,,,,,,,,,,,,,,
Atlantic,0,0,0,6,1,1,1,0,3,2,...,1,1,0,0,2,1,2,1,0,2
Breitbart,4,0,4,0,4,8,2,6,11,17,...,1,1,3,1,3,29,13,14,9,8
Business Insider,1,1,1,1,0,7,1,1,2,6,...,4,1,1,1,0,4,1,3,0,2
Buzzfeed News,2,0,0,0,1,0,0,0,1,0,...,0,6,1,0,1,0,0,0,0,0
CNN,2,0,5,3,0,8,1,0,4,5,...,1,0,0,0,0,7,3,1,0,4
Fox News,4,0,1,0,0,1,0,1,3,7,...,0,0,1,0,1,6,5,5,0,3
Guardian,0,0,1,1,2,0,1,0,1,0,...,1,0,3,0,1,2,1,0,0,1
NPR,2,0,2,4,4,4,2,1,3,3,...,1,0,1,0,2,6,1,0,1,0
National Review,0,0,1,0,0,0,2,1,1,5,...,0,0,0,1,1,1,1,3,5,3


In [26]:
print('Adjusted Rand Score: {:0.7}'.format(adjusted_rand_score(y_train, y_pred3)))
print('Silhouette Score: {:0.7}'.format(silhouette_score(features, y_pred3, sample_size=60000, metric='euclidean')))

Adjusted Rand Score: 0.004529963
Silhouette Score: -0.03694141


And the results are worthless, just pitiful. Seems like k-means is the best clustering algorithm- mostly because our other methods were far worse, not because it performed well.

In [41]:
X_train_cluster = pd.DataFrame(features)
X_train_cluster['kmeans'] = y_pred

# Training the Model

So now that we attempted clustering with the datset, it's time to run the models.

### Random Forest

In [43]:
rfc = ensemble.RandomForestClassifier()
rfc_train = cross_val_score(rfc, features, y_train, cv=5, n_jobs=-1)
print('Random forest classifier score (without clustering): {:.5f}(+/- {:.2f})\n'.format(rfc_train.mean(), rfc_train.std()*2))

rfc_train_c = cross_val_score(rfc, X_train_cluster, y_train, cv=5, n_jobs=-1)
print('Random forest classifier score (with clustering): {:.5f}(+/- {:.2f})'.format(rfc_train_c.mean(), rfc_train_c.std()*2))

Random forest classifier score (without clustering): 0.40743(+/- 0.03)

Random forest classifier score (with clustering): 0.41103(+/- 0.03)


### Logistic Regression

In [46]:
lr = LogisticRegression()
lr_train = cross_val_score(lr, features, y_train, cv=5, n_jobs=-1)
print('Logistic regression score (without clustering): {:.5f}(+/- {:.2f})\n'.format(lr_train.mean(), lr_train.std()*2))

lr_train_c = cross_val_score(lr, X_train_cluster, y_train, cv=5, n_jobs=-1)
print('Logistic regression score (with clustering): {:.5f}(+/- {:.2f})'.format(lr_train_c.mean(), lr_train_c.std()*2))

Logistic regression score (without clustering): 0.43547(+/- 0.03)

Logistic regression score (with clustering): 0.43547(+/- 0.03)


### Gradient Boosting Classifier

In [47]:
gbc = ensemble.GradientBoostingClassifier()
gbc_train = cross_val_score(gbc, features, y_train, cv=5, n_jobs=-1)
print('Gradient boosting classifier score (without clustering): {:.5f}(+/- {:.2f})\n'.format(gbc_train.mean(), gbc_train.std()*2))

gbc_train_c = cross_val_score(gbc, X_train_cluster, y_train, cv=5, n_jobs=-1)
print('Gradient boosting classifier score (with clustering): {:.5f}(+/- {:.2f})'.format(gbc_train_c.mean(), gbc_train_c.std()*2))

Gradient boosting classifier score (without clustering): 0.48250(+/- 0.02)

Gradient boosting classifier score (with clustering): 0.48078(+/- 0.02)


### Optimized Gradient Boosting Classifier 

In [30]:
# Parameters for gradient boosting classifier
param_grid  = {'loss':['deviance'],
               'max_features': ['sqrt'],
               'n_estimators': [400, 800],
               'max_depth': [12, 20],
               "min_samples_leaf" : [12, 20]}

# Run grid search to find ideal parameters
gbc_grid = GridSearchCV(gbc, param_grid = param_grid, n_jobs=-1)

# Initialize and fit the model.
gbc_grid.fit(features, y_train)

# Return best parameters and best score
print('Best parameters:')
print(gbc_grid.best_params_)
print('Best Score:')
print(gbc_grid.best_score_)

Best parameters:
{'loss': 'deviance', 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 20, 'n_estimators': 800}
Best Score:
0.4993566833452466


# Testing the Model

In [31]:
# Normalize Tf-idf vectors
X_test_norm = normalize(X_test_tfidf)

In [32]:
X_test_words = []

for row in X_test:
    # Processing each row for tokens
    row_doc = nlp(row)
    # Calculating length of each sentence
    sent_len = len(row_doc) 
    # Initializing counts of different parts of speech
    advs = 0
    verb = 0
    noun = 0
    adj = 0
    for token in row_doc:
        # Identifying each part of speech and adding to counts
        if token.pos_ == 'ADV':
            advs +=1
        elif token.pos_ == 'VERB':
            verb +=1
        elif token.pos_ == 'NOUN':
            noun +=1
        elif token.pos_ == 'ADJ':
            adj +=1
    # Creating a list of all features for each sentence
    X_test_words.append([row_doc, advs, verb, noun, adj, sent_len])
    
# Data frame for features
X_test_count = pd.DataFrame(data=X_test_words, columns=['BOW', 'ADV', 'VERB', 'NOUN', 'ADJ', 'sent_length'])

# Change token count to token percentage
for column in X_test_count.columns[1:5]:
    X_test_count[column] = X_test_count[column] / X_test_count['sent_length']

# Normalize X_count
X_test_counter = normalize(X_test_count.drop('BOW',axis=1))
X_test_counter  = pd.DataFrame(data=X_test_counter)

In [33]:
# Combining features into one data frame
X_test_norm_df = pd.DataFrame(data=X_test_norm.toarray())
features_test = pd.concat([X_test_counter, X_test_norm_df], ignore_index=False, axis=1)
features_test.head()

,0,1,2,3,4,0,1,2,3,4,...,140,141,142,143,144,145,146,147,148,149
0,0.000212,0.000750,0.000846,0.000231,0.999999,0.000000,0.000000,0.000000,0.474484,0.000000,...,0.168367,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,0.000047,0.000186,0.000221,0.000078,1.000000,0.182195,0.000000,0.000000,0.037380,0.000000,...,0.079584,0.085011,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.090096
2,0.000028,0.000097,0.000143,0.000049,1.000000,0.099928,0.052946,0.000000,0.000000,0.053769,...,0.087298,0.046625,0.000000,0.051634,0.136435,0.0,0.164890,0.118228,0.150292,0.049415
3,0.000090,0.000368,0.000269,0.000099,1.000000,0.000000,0.000000,0.092247,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.151133,0.072046,0.000000
4,0.000034,0.000109,0.000109,0.000049,1.000000,0.058145,0.000000,0.000000,0.000000,0.062572,...,0.000000,0.054259,0.456313,0.000000,0.000000,0.0,0.063963,0.000000,0.000000,0.000000


In [34]:
# Calulate predicted values
kmeans = KMeans(n_clusters=15, init='k-means++', random_state=42, n_init=20)
y_pred_test = kmeans.fit_predict(features_test)

pd.crosstab(y_test, y_pred_test)

col_0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
publication,,,,,,,,,,,,,,,
Atlantic,3,19,26,2,10,17,9,4,0,3,5,6,2,12,2
Breitbart,8,36,89,37,58,43,79,18,26,2,30,12,9,78,27
Business Insider,0,26,21,4,8,7,3,6,1,0,6,2,17,34,3
Buzzfeed News,2,9,6,9,2,2,10,5,0,0,7,0,7,14,2
CNN,7,17,27,17,9,20,6,12,2,4,17,1,1,44,9
Fox News,0,3,9,12,15,6,3,2,4,3,3,0,3,20,2
Guardian,3,21,13,8,3,7,4,1,0,0,7,0,4,13,3
NPR,9,39,19,5,5,17,4,6,2,47,13,4,6,22,6
National Review,4,17,27,0,13,5,3,1,3,0,11,4,0,7,6


In [35]:
print('Adjusted Rand Score: {:0.7}'.format(adjusted_rand_score(y_test, y_pred_test)))
print('Silhouette Score: {:0.7}'.format(silhouette_score(features_test, y_pred_test, sample_size=60000, metric='euclidean')))

Adjusted Rand Score: 0.02438879
Silhouette Score: 0.07111983


In [36]:
X2_test_c = pd.DataFrame(features_test)
X2_test_c['kmeans_clust'] = y_pred_test

In [37]:
gbc_grid_scores_test = cross_val_score(gbc_grid, features_test, y_test, cv=5)
print('Test set score: {:.5f}(+/- {:.3f})'.format(gbc_grid_scores_test.mean(), gbc_grid_scores_test.std()*2))

Test set score: 0.46362(+/- 0.034)


# Conclusion

# Source

https://www.kaggle.com/snapcrack/all-the-news